<H2>Pré requisitos para rodar o codigo</H2>

Ter o ollama instalado na maquina para usar o modelo Gemma2:2b

<ol>
    <li>Baixe o instalador do <a href="https://ollama.com/download">site oficial</a> </li>
    <li>Execute o arquivo .exe e siga as instruções do assistente de instalação.</li>
    <li>Após a instalação, abra o terminal (cmd ou PowerShell) e verifique se Ollama está instalado corretamente com o comando ollama --version.</li>
    <li>No terminal (cmd ou PowerShell) de o comando "ollama pull gemma2:2b" para baixar o modelo </li>
    <li>Asimm que finalizar o downlaod de o comando "ollama pull nomic-embed-text" que vamos usar para gerar embeddings</li>
</ol>

Instalações

<p>Comando usando o conda <br>
    conda install Langchain-community langachain-experimental PyPDF sentence-transformers chromadb langchain_ollama
</p>
<p>Comando usando o pip <br>
    pip install Langchain-community langachain-experimental PyPDF sentence-transformers chromadb langchain_ollama
</p>


<h3>Bibliotecas usadas</H3>
<h4>Langchain</H4>
<p>Langchain é uma biblioteca de Python que facilita a criação de aplicações de processamento de linguagem natural (NLP) complexas e personalizadas. Ela oferece uma interface intuitiva para conectar diferentes componentes de NLP, como modelos de linguagem, vetores, e fontes de dados. A documentação pode ser encontrada <a href="https://python.langchain.com/docs/introduction/">clicando aqui</a>  </p>
<h4>Transformers </H4>
<p>Transformers é uma arquitetura de rede neural que revolucionou o campo do NLP. Modelos como GPT-3 e BERT são baseados em transformers e são capazes de realizar tarefas complexas como tradução de idiomas, geração de texto e resposta a perguntas. <a href="https://huggingface.co/docs/transformers/index">clicando aqui</a>  </p>
<h4>Chromadb</H4>
<p>Chroma é um banco de dados de vetores especialmente projetado para aplicações de IA. Ele armazena e recupera embeddings (representações numéricas de texto) de forma eficiente, o que é fundamental para tarefas como busca semântica e recomendação. <a href="https://docs.trychroma.com/">clicando aqui</a>  </p>
<h4>Ollama</H4>
<p>Ollama é uma ferramenta que permite rodar modelos de linguagem de grande escala (LLMs) localmente, no próprio computador. Isso significa que podemos utilizar modelos como gemma 2 e o nomic-embed-text sem depender de APIs externas, o que pode reduzir custos e aumentar a privacidade dos dados. A documentação que podemos seguir é a que esta no langchain ja que vamos integrar o modelo a essa biblioteca, a documentação pode ser acessada <a href="https://python.langchain.com/api_reference/ollama/index.html">clicando aqui</a></p>


<H5>0 - Defininfo o modelo como o gemma2 de 2 bilões de parametros</H5>

In [3]:
from langchain_community.chat_models import ChatOllama
model = ChatOllama(model='gemma2:2b')

<H5>1 - Extração e tratamento de Dados</H5>
<ul>
<li>loader.load() - Carrega o texto conforme o caminho passado</li>
<li>HuggingFaceEmbeddings() - Gera embeddings</li>
<li>SemanticChunker()- dividir textos de forma semantica.
    <ul>
        <li>breakpoint_threshold_type - decidir onde o texto sera dividido nesse caso foi definido como "standard_deviation" significa que o algoritmo ira analisar a similaridade semantica entre as partes do texto e identificar os pontos onde essa   similaridade muda significativamente.</li>
    </ul>
 </li>
<li>split_documents() - divide o documento em partes menores</li>
</ul>





In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Caminho do PDF")

docs_split = loader.load()
hf_embeddings = HuggingFaceEmbeddings()
text_splitter = SemanticChunker(hf_embeddings, breakpoint_threshold_type="standard_deviation") 
chunks = text_splitter.split_documents(docs_split)


<H5>2 - Convertendo chunks para embeddings e armarmazenando</H5>
<ul>
    <li>
        Chroma.from_documents() - cria a vector store
        <ul>
            <li>documents - passa os chunks de texto gerados na celula anterior</li>
            <li>embedding - define que os embeddings seraão gerados pelo modelo nomic-embed-text</li>
        </ul>
    </li>
    <li>retriever = usado para buscar os documentos mais relevantes dentro do banco de dados, com base em uma query</li>
</ul>

    
    


In [5]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model='nomic-embed-text')
)
retriever = vectorstore.as_retriever()

<H5>3 - Fazendo a pergunta</H5>
<ul>
    <li>prompt_templat - Define o formato da pergunta que será feita ao modelo de linguagem.</li>
    <li>ChatPromptTemplate - Cria um objeto ChatPromptTemplate a partir do template definido.
        <ul>
            <li>from_template() - define o template da pergunta que será respondida</li>
        </ul>
    </li>
    <li>chain - Define a cadeia de passos que o modelo seguirá para responder à pergunta.
        <ul>
            <li>retriever - Busca o contexto relevante para a pergunta.</li>
            <li>RunnablePassthrough() - Passa a pergunta diretamente para o próximo passo.</li>
            <li>prompt - Formata a pergunta final usando o template e o contexto.</li>
            <li>model - O modelo de linguagem processa a pergunta formatada e gera a resposta.</li>
            <li>StrOutputParser() - Converte a saída do modelo em uma string.</li>
        </ul>
    </li>
    <li>invoke - Executa a cadeia de passos definida e imprime a resposta gerada pelo modelo.</li>
</ul>


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt_templat = """Responda à pergunta com base apenas no seguinte contexto:
{context}
Pergunta: {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_templat)
chain = ({'context': retriever, 'question': RunnablePassthrough()} |
                   prompt | model | StrOutputParser())
while True:
    question = input("Digite a pergunta : ")
    print("Pensando:\n")
    print(chain.invoke(question))
